In [2]:
import gdal
import pandas as pd
import numpy as np

In [3]:
def get_val(dat,gt=[],rb=[]):
    ''' Grab cell value based on data frame x and y variables.
    dat = dataframe containing x and y 
    gt = geotransform, of the index raster
    rb = raster band to pick data from
    '''
    mx = dat.x
    my = dat.y
    # transform to array coordinates
    px = int((mx-gt[0])/gt[1]) # x pixle
    py = int((my-gt[3])/gt[5]) # y pixle

    # extract the value
    intval = rb.ReadAsArray(px,py,1,1)
    return intval[0][0]

In [5]:
fl = './data/CPG_from_roy/region_17/MirAd_2012_100_CPG.tif'
ds = gdal.Open(fl)
gt = ds.GetGeoTransform()
rb = ds.GetRasterBand(1)

In [9]:
fl = '../DEM_processing/data/CATCHMENT_reg_17_snapped_ID.csv'
testDat = pd.read_csv(fl)
del testDat['Unnamed: 0']
del testDat['Unnamed: 0.1']

In [10]:
testDat.head()

,cat,x,y,Gage_no
0,1,-1703250.0,2521380.0,13220500
1,2,-2192520.0,2442660.0,14362250
2,3,-2119230.0,2708040.0,14173500
3,4,-2093280.0,2594610.0,14145100
4,5,-2181330.0,2693610.0,14306400
